<a href="https://colab.research.google.com/github/ggvaldez/LGPD-Script/blob/main/LGPD_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>